In [13]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.01):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    if total_rule/total_records<0.4:
        return (999,);

    avg_confidence = np.sum(rule_confidences) / count +1*(entropy_D-entropy_DB)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    return hof



def bestIndividual(hof,df):

    def SVM(trainX,trainY,testX,testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=200)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY)/testY.shape[0]
        return precision
    maxaccuracy=0.0
    for ind in hof:
        df = pd.read_csv('data/chess/chess.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.33,random_state=1)
        score=SVM(trainX,trainY,testX,testY)
        if(score>maxaccuracy):
            maxaccuracy=score
            print(maxaccuracy)
            best_feature=selected_features
            print(best_feature)
    return maxaccuracy,best_feature


if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=50
    n_gen=100
    # read dataframe from csv
    df = pd.read_csv('data/chess/chess.csv')
    # encode labels column to numbers
    y = df['target']
    feature_columns = [ col for col in df.columns if col not in ['id','target']]
    X = df[feature_columns]
    individual = [1 for i in range(len(X.columns))]
    getFitness(df,'target',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'target',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'target',n_pop, n_gen)

Accuracy with all features: 	(999,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min    
0  	50    	899.303	1.62187
1  	32    	759.717	1.612  
2  	33    	520.456	1.612  
3  	26    	281.167	1.612  
4  	21    	61.6789	1.54726
5  	27    	121.359	1.17499
6  	29    	221.055	1.53253
7  	31    	141.23 	1.22857
8  	39    	101.273	1.13606
9  	35    	101.219	1.13072
10 	36    	101.118	1.12743
11 	25    	81.0966	1.12743
12 	30    	21.1435	1.12743
13 	26    	140.837	1.12737
14 	37    	100.927	1.12737
15 	31    	61.0109	1.12683
16 	32    	80.9664	1.12681
17 	32    	61.0153	1.12675
18 	28    	80.9605	1.12662
19 	39    	80.9694	1.12662
20 	27    	41.0516	1.12662
21 	33    	100.916	1.11706
22 	27    	41.0448	1.11643
23 	33    	120.902	1.11643
24 	31    	100.937	1.11629
25 	28    	120.878	1.11629
26 	22    	1.13167	1.11629
27 	33    	120.865	1.11629
28 	30    	41.032 	1.07491
29 	22    	60.9901	1.07491
30 	35    	200.71 	1.07491
31 	32    	60.9962	1.07417
32 	32    	80.9402	1.07417
33 	32    	140.816	1.07417
34 	32    	140.791	1.07417
35 	36    	180.713	1.07417
3

In [14]:
accuracy,individual = bestIndividual(hof,df)
print(accuracy)
print(individual)
    # select the best individual

    # print('Best Accuracy: \t' + str(accuracy))
    # print('Number of Features in Subset: \t' + str(individual.count(1)))
    # print('Individual: \t\t' + str(individual))
    # print('Feature Subset\t: ' + str(header))
    #
    # print('\n\ncreating a new classifier with the result')

    # read dataframe from csv one more time
    # df = pd.read_csv(dataframePath, sep=',')
    #
    # # with feature subset
    # X = df[header]
    #
    # clf = LogisticRegression()
    #
    # scores = cross_val_score(clf, X, y, cv=5)
    # print("Accuracy with Feature Subset: \t" + str(avg(scores)) + "\n")


0.9677725118483412
['f1', 'f2', 'f6', 'f8', 'f9', 'f10', 'f14', 'f15', 'f16', 'f17', 'f19', 'f20', 'f21', 'f23', 'f25', 'f28', 'f29', 'f30', 'f32', 'f33', 'f35']
0.9687203791469194
['f1', 'f2', 'f6', 'f8', 'f9', 'f10', 'f14', 'f15', 'f16', 'f17', 'f19', 'f20', 'f21', 'f23', 'f25', 'f28', 'f30', 'f32', 'f33', 'f35']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.9706161137440759
['f1', 'f2', 'f6', 'f8', 'f9', 'f10', 'f14', 'f15', 'f16', 'f17', 'f20', 'f21', 'f23', 'f25', 'f28', 'f30', 'f32', 'f33', 'f35']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.9734597156398104
['f1', 'f2', 'f6', 'f8', 'f9', 'f10', 'f14', 'f15', 'f16', 'f17', 'f19', 'f21', 'f23', 'f25', 'f28', 'f30', 'f32', 'f33', 'f35']
0.9753554502369668
['f1', 'f2', 'f6', 'f8', 'f10', 'f14', 'f15', 'f16', 'f17', 'f19', 'f20', 'f21', 'f23', 'f25', 'f28', 'f30', 'f32', 'f33', 'f35']


...

0.9781990521327014
['f1', 'f2', 'f6', 'f9', 'f10', 'f14', 'f15', 'f16', 'f17', 'f19', 'f21', 'f23', 'f25', 'f28', 'f32', 'f33', 'f35']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.9791469194312796
['f1', 'f2', 'f6', 'f7', 'f8', 'f10', 'f14', 'f15', 'f16', 'f17', 'f20', 'f21', 'f23', 'f25', 'f28', 'f29', 'f30', 'f32', 'f33', 'f35']


C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\svm\_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\sklearn\sv

0.9791469194312796
['f1', 'f2', 'f6', 'f7', 'f8', 'f10', 'f14', 'f15', 'f16', 'f17', 'f20', 'f21', 'f23', 'f25', 'f28', 'f29', 'f30', 'f32', 'f33', 'f35']
